In [2]:
import numpy as np
from numba import jit, vectorize, float64, int64

## This is one dimensional comparison
### Assume gradient is known
The posterior follows normal distribution with zero mean and covariance matrix:
$
\begin{bmatrix} 
1 & 0.9& 0.8\\
0.9 & 1 &0.9\\
0.8 &0.9&1\\
\end{bmatrix}
$

In [4]:
@jit
def grad(theta):
    """This is the gradient function"""
    return np.linalg.inv(np.array([[1,.9,0.8],[.9,1,0.9],[0.8,0.9,1]])) @ theta

### Regular Python

In [17]:
def hmcvec(grad,theta0,M,C,epsilon,iter=1000):
    """
    This function outputs the p dimiension Hamilton Monte Carlo samples without M-H correction.    
    Args: 
        grad: the gradient of the potential energy
        theta0: the initial point of theta, the parameter of interest
        M: the mass
        C: the C term, where C*M^{-1} is the friction
        epsilon: stepsize
        p: the dimension of theta
        iter: iteration number, 1000 by default
    """
    p=theta0.shape[0]
    r=np.random.multivariate_normal(np.zeros(p),M)
    theta=theta0
    theta_save=np.zeros([iter,p])
    r_save=np.zeros([iter,p])
    for t in range(iter):    
        theta=theta+epsilon*np.linalg.inv(M)@r
        r=r-grad1(theta)*epsilon-epsilon*(np.linalg.inv(M)@r)+np.random.multivariate_normal(np.zeros(p),2*epsilon*C,1).ravel()
        theta_save[t,:]=theta
        r_save[t,:]=r
    return np.c_[theta_save,r_save]

In [18]:
#check result
theta0=np.zeros(3)
M=np.eye(3)
C=np.eye(3)
epsilon=.1
iter=30000
out=hmcvec(grad,theta0,M,C,epsilon,iter=30000)
np.corrcoef([out[:,0],out[:,1],out[:,2]])

array([[1.        , 0.89786766, 0.80310328],
       [0.89786766, 1.        , 0.90023317],
       [0.80310328, 0.90023317, 1.        ]])

In [19]:
%timeit hmcvec(grad,theta0,M,C,epsilon,iter=30000)

9.08 s ± 32.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numba Version

In [22]:
@jit([float64[:,:](float64[:,:],float64[:],float64[:,:],float64[:,:],float64, int64)],cache=True)
def hmc_nbvec(grad,theta0,M,C,epsilon,iter=1000):
    """
    This function outputs the p dimiension Hamilton Monte Carlo samples without M-H correction.    
    Args: 
        theta0: the initial point of theta, the parameter of interest
        grad: the gradient of the potential
        M: the mass
        C: the C term, where C*M^{-1} is the friction
        epsilon: stepsize
        p: the dimension of theta
        iter: iteration number, 1000 by default
    """
    
    p=theta0.shape[0]
    r=np.random.multivariate_normal(np.zeros(p),M)
    theta=theta0
    theta_save=np.zeros([iter,p])
    r_save=np.zeros([iter,p])
    for t in range(iter):    
        mr=np.linalg.solve(M,r)
        theta=theta+epsilon*mr
        r=r-grad1(theta)*epsilon-epsilon*np.dot(C,mr)+np.random.multivariate_normal(np.zeros(p),2*epsilon*C,1).ravel()
        theta_save[t,:]=theta
        r_save[t,:]=r
    return np.c_[theta_save,r_save]

In [23]:
#check result
theta0=np.zeros(3)
M=np.eye(3)
C=np.eye(3)
epsilon=.1
iter=30000
out=hmc_nbvec(grad,theta0,M,C,epsilon,iter=30000)
np.corrcoef([out[:,0],out[:,1],out[:,2]])

array([[1.        , 0.90167228, 0.80501729],
       [0.90167228, 1.        , 0.9027092 ],
       [0.80501729, 0.9027092 , 1.        ]])

In [24]:
%timeit hmc_nbvec(grad,theta0,M,C,epsilon,iter=30000)

7.52 s ± 49.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
